## Ai_wolf (Gemini Financial Analysis Assistant) Colab Launcher

這個 Notebook 用於在 Google Colab 環境中快速啟動 Ai_wolf 專案。

**使用步驟：**
1.  **設定 API 金鑰**：
    *   在 Colab 的左側邊欄中，點擊「金鑰」(Secrets) 圖示。
    *   新增以下兩個金鑰：
        *   `GOOGLE_API_KEY`：填入您的 Google Gemini API 金鑰。
        *   `FRED_API_KEY`：填入您的 FRED API 金鑰。
    *   確保「Notebook access」已啟用。
2.  **執行儲存格**：
    *   點擊 "Runtime" -> "Run all"，或按順序執行以下兩個 Code Cell。
3.  **訪問應用程式**：
    *   當 Code Cell 2 執行時，它會安裝 `cloudflared` 並啟動 Streamlit 應用程式。
    *   在 Cell 2 的輸出中，尋找類似 `https://<your-tunnel-name>.trycloudflare.com` 的網址。
    *   點擊該網址即可在瀏覽器中打開 Ai_wolf 應用程式。
4.  **日誌**：
    *   Streamlit 應用程式的標準輸出和錯誤將直接顯示在 Code Cell 2 的輸出區域。
    *   應用程式的詳細日誌 (由 `app.py` 中的日誌系統配置) 會寫入到您的 Google Drive 中的 `/content/drive/MyDrive/MyWolfData/logs/streamlit.log` (如果 Drive 掛載成功) 或 Colab 本地儲存的 `/content/MyWolfData/logs/streamlit.log`。

In [ ]:
import shutil
import os

project_dir_name = "wolfAI" # 專案的資料夾名稱
default_colab_project_path = f'/content/{project_dir_name}' # Colab 本地儲存中的專案路徑

# 決定專案路徑 (優先使用 GDrive)
GDRIVE_PROJECT_DIR = default_colab_project_path # 預設為本地路徑
IS_GDRIVE_MOUNTED = False

print("--- 環境準備與專案下載 ---")

# 1. 清理舊的專案資料夾 (如果存在)
print(f"檢查舊的專案資料夾 '{default_colab_project_path}' (本地) 和 '/content/drive/MyDrive/{project_dir_name}' (GDrive)...")
paths_to_clean = [default_colab_project_path, f'/content/drive/MyDrive/{project_dir_name}']
for path_to_clean in paths_to_clean:
    if os.path.exists(path_to_clean):
        print(f"警告：偵測到已存在的 '{path_to_clean}' 資料夾。執行強制清理...")
        try:
            shutil.rmtree(path_to_clean)
            print(f"'{path_to_clean}' 已成功刪除。")
        except Exception as e:
            print(f"刪除 '{path_to_clean}' 時發生錯誤: {e}")
            print("請手動檢查並刪除該資料夾，然後重新執行此儲存格。")
            raise
    else:
        print(f"'{path_to_clean}' 不存在，無需清理。")

# 2. 嘗試掛載 Google Drive 並設定資料路徑
data_dir_gdrive_base = '/content/drive/MyDrive/MyWolfData'
data_dir_local_base = '/content/MyWolfData'
log_dir_to_create = ''

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IS_GDRIVE_MOUNTED = True
    print(f"Google Drive 掛載成功。專案將嘗試使用 GDrive 路徑。")
    GDRIVE_PROJECT_DIR = f'/content/drive/MyDrive/{project_dir_name}' # GDrive 中的專案路徑
    log_dir_to_create = os.path.join(data_dir_gdrive_base, 'logs')
except ImportError:
    print("未偵測到 Google Colab 環境或 google.colab 模組導入失敗。假設為本地執行模式。")
    log_dir_to_create = os.path.join(data_dir_local_base, 'logs')
    GDRIVE_PROJECT_DIR = default_colab_project_path # 回退到 Colab 本地儲存
except Exception as e:
    print(f"Google Drive 掛載時發生錯誤: {e}")
    print("將回退到使用 Colab 本地儲存路徑。")
    log_dir_to_create = os.path.join(data_dir_local_base, 'logs')
    GDRIVE_PROJECT_DIR = default_colab_project_path # 回退到 Colab 本地儲存

# 3. 創建 MyWolfData/logs 資料夾 (根據 Drive 是否掛載成功)
print(f"目標日誌資料夾路徑: '{log_dir_to_create}'")
if not os.path.exists(log_dir_to_create):
    os.makedirs(log_dir_to_create, exist_ok=True)
    print(f"日誌資料夾 '{log_dir_to_create}' 已創建。")
else:
    print(f"日誌資料夾 '{log_dir_to_create}' 已存在。")

# 4. 克隆專案到目標路徑
print(f"專案將被克隆到: {GDRIVE_PROJECT_DIR}")
# 確保目標克隆路徑的上層目錄存在 (例如 /content/drive/MyDrive/ 如果 GDRIVE_PROJECT_DIR 在其中)
project_parent_dir = os.path.dirname(GDRIVE_PROJECT_DIR)
if not os.path.exists(project_parent_dir) and IS_GDRIVE_MOUNTED and "/content/drive/MyDrive" in project_parent_dir:
     # 如果父目錄是 MyDrive 下的某個目錄且不存在，這通常不應該發生，除非 MyDrive 本身有問題
     print(f"警告：專案的父目錄 '{project_parent_dir}' 在已掛載的 Drive 中不存在。Git clone 可能會失敗。")
elif not os.path.exists(project_parent_dir) and not IS_GDRIVE_MOUNTED and "/content" in project_parent_dir:
     # 對於本地 /content 下的路徑，父目錄 /content 應該永遠存在
     pass 
elif not os.path.exists(project_parent_dir):
    try:
        os.makedirs(project_parent_dir, exist_ok=True)
        print(f"已創建專案的父目錄 '{project_parent_dir}'。")
    except Exception as e_mkdir_parent:
        print(f"創建專案父目錄 '{project_parent_dir}' 失敗: {e_mkdir_parent}。Git clone 可能會失敗。")

print("正在克隆專案 (使用 --depth 1 精簡下載)...")
# 使用 subprocess 執行 git clone，以便更好地處理路徑中的空格和特殊字符
import subprocess
git_clone_command = ['git', 'clone', '--depth', '1', 'https://github.com/LinWolf/Gemini_Financial_Analysis_Assistant.git', GDRIVE_PROJECT_DIR]
clone_process = subprocess.run(git_clone_command, capture_output=True, text=True)
if clone_process.returncode == 0:
    print(f"專案成功克隆到 '{GDRIVE_PROJECT_DIR}'。")
    print(clone_process.stdout)
else:
    print(f"錯誤：克隆專案到 '{GDRIVE_PROJECT_DIR}' 失敗。")
    print(f"Git Stderr: {clone_process.stderr}")
    print(f"Git Stdout: {clone_process.stdout}")
    raise ChildProcessError(f"Git clone 失敗，無法繼續。錯誤: {clone_process.stderr}")

# 5. 安裝依賴套件
requirements_file_path = os.path.join(GDRIVE_PROJECT_DIR, "requirements.txt")
if not os.path.isfile(requirements_file_path):
    print(f"錯誤：找不到 requirements.txt 於 '{requirements_file_path}'。無法安裝依賴。")
    raise FileNotFoundError(f"找不到 requirements.txt 於 '{requirements_file_path}'")

print(f"正在從 '{requirements_file_path}' 安裝依賴套件...")
# 使用 subprocess 執行 pip install
pip_install_command = ['pip', 'install', '-q', '-r', requirements_file_path]
install_process = subprocess.run(pip_install_command, capture_output=True, text=True)
if install_process.returncode == 0:
    print("依賴套件安裝完成。")
    if install_process.stdout:
        print(f"Pip Stdout:\n{install_process.stdout}")
else:
    print(f"錯誤：依賴套件安裝失敗。Return code: {install_process.returncode}")
    if install_process.stderr:
        print(f"Pip Stderr:\n{install_process.stderr}")
    if install_process.stdout:
        print(f"Pip Stdout:\n{install_process.stdout}")
    raise ChildProcessError(f"Pip install 失敗。錯誤: {install_process.stderr or install_process.stdout}")

print("--- 環境準備完畢 --- KERNEL_RESTART_REQUIRED_PLEASE_RE-RUN_NEXT_CELL --- ")
# 提示用戶：由於 pip install 可能安裝了新的套件或更新了現有套件，
# Colab 環境可能需要重啟 Kernel 以使新套件生效。
# 但我們這裡不強制重啟，而是讓用戶手動執行下一個 cell。
# 如果 streamlit 啟動失敗，用戶應嘗試 "Runtime" -> "Restart session" 然後僅執行下一個 cell。

In [ ]:
import os
import subprocess
import threading
import time
import sys
import signal # 用於更優雅地處理中斷

print("--- 驗證與啟動應用程式 ---")

# 1. 重新確定專案目錄路徑 (GDRIVE_PROJECT_DIR)
# 確保此邏輯與上一個儲存格中的 GDRIVE_PROJECT_DIR 設定方式一致
project_dir_name = "wolfAI"
gdrive_potential_path = f"/content/drive/MyDrive/{project_dir_name}"
local_potential_path = f"/content/{project_dir_name}"
ACTUAL_PROJECT_DIR = ""

if os.path.exists(gdrive_potential_path):
    ACTUAL_PROJECT_DIR = gdrive_potential_path
    print(f"偵測到專案目錄於 Google Drive: {ACTUAL_PROJECT_DIR}")
elif os.path.exists(local_potential_path):
    ACTUAL_PROJECT_DIR = local_potential_path
    print(f"偵測到專案目錄於本地 Colab 儲存: {ACTUAL_PROJECT_DIR}")
else:
    print(f"錯誤：專案目錄在 '{gdrive_potential_path}' 或 '{local_potential_path}' 中均未找到。")
    print("請確保前一個儲存格已成功執行，並且專案已克隆到預期位置。")
    print("如果前一個儲存格提示需要重啟 Kernel，請先執行 Runtime -> Restart session，然後重新運行此儲存格。")
    raise FileNotFoundError("專案目錄未找到，無法啟動應用程式。")

APP_PY_PATH = os.path.join(ACTUAL_PROJECT_DIR, "app.py")
print(f"預期 app.py 路徑: {APP_PY_PATH}")

if not os.path.isfile(APP_PY_PATH):
    print(f"錯誤：核心檔案 'app.py' 在 '{APP_PY_PATH}' 未找到。部署可能失敗。")
    print("請檢查 GitHub 倉庫結構和克隆過程是否正確。")
    raise FileNotFoundError(f"app.py 未在 '{APP_PY_PATH}' 找到。")

print(f"'app.py' 存在。準備啟動 Streamlit 服務從 '{ACTUAL_PROJECT_DIR}'...")
print("Streamlit 的標準輸出和錯誤將顯示在此儲存格下方。")
print("應用程式日誌 (由 app.py 配置) 將寫入 MyWolfData/logs/streamlit.log (如果配置正確)。")

# 2. 安裝/檢查 cloudflared
print("正在檢查/安裝 cloudflared...")
try:
    # 嘗試獲取 cloudflared 版本，如果成功則表示已安裝
    cf_version_check = subprocess.run(['cloudflared', '--version'], capture_output=True, text=True, check=False)
    if cf_version_check.returncode == 0 and "cloudflared version" in cf_version_check.stdout:
        print(f"cloudflared 已安裝: {cf_version_check.stdout.strip()}")
    else:
        raise FileNotFoundError # 觸發安裝
except FileNotFoundError:
    print("cloudflared 未找到，正在嘗試安裝...")
    npm_install_process = subprocess.run(['npm', 'install', '-g', 'cloudflared'], capture_output=True, text=True, check=False)
    if npm_install_process.returncode == 0:
        print("cloudflared 安裝成功 (via npm)。")
        # 驗證安裝
        cf_version_check_after_install = subprocess.run(['cloudflared', '--version'], capture_output=True, text=True, check=False)
        if cf_version_check_after_install.returncode == 0:
             print(f"cloudflared 版本: {cf_version_check_after_install.stdout.strip()}")
        else:
            print(f"cloudflared 安裝後版本檢查失敗: {cf_version_check_after_install.stderr}")
            # raise ChildProcessError("cloudflared 安裝後版本檢查失敗。請檢查 npm 是否正確安裝與配置於 PATH。"))
            # 即使版本檢查失敗，也嘗試繼續，因為 npm install 可能沒有立即更新 PATH 給 subprocess
    else:
        print(f"cloudflared 安裝失敗 (npm)。錯誤: {npm_install_process.stderr}")
        print("警告：無法安裝 cloudflared。應用程式可能無法從外部訪問。")
        # 可以考慮後備到 ngrok 或不啟動隧道

# 3. 啟動 Streamlit 應用程式和 cloudflared 隧道
streamlit_process = None
tunnel_process = None
stop_event = threading.Event() # 用於通知線程停止

def stream_output(pipe, label, is_tunnel_err_pipe=False):
    try:
        for line in iter(pipe.readline, ''):
            if stop_event.is_set(): break
            line_stripped = line.strip()
            print(f"[{label}] {line_stripped}", flush=True)
            # cloudflared 的 URL 通常在其 stderr 中，且包含 .trycloudflare.com
            if is_tunnel_err_pipe and ".trycloudflare.com" in line_stripped and "http" in line_stripped:
                url_start_index = line_stripped.find('http')
                url_candidate = line_stripped[url_start_index:].split(' ')[0]
                # 移除 ANSI escape codes (常見於 cloudflared 輸出)
                import re
                ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
                url_candidate = ansi_escape.sub('', url_candidate)
                if url_candidate.startswith("https") and ".trycloudflare.com" in url_candidate:
                    print(f"\n🚀🚀🚀 您的應用程式應該很快就可以在以下公開網址訪問： {url_candidate} 🚀🚀🚀\n")
    except Exception as e:
        if not stop_event.is_set(): # 只在不是因為 stop_event 導致的退出時打印錯誤
             print(f"輸出串流錯誤 ({label}): {e}", flush=True)
    finally:
        pipe.close()
        print(f"[{label}] 輸出串流已關閉。", flush=True)

try:
    print("正在啟動 Streamlit 應用程式...")
    streamlit_process = subprocess.Popen(
        [sys.executable, "-m", "streamlit", "run", APP_PY_PATH, 
         "--server.port", "8501", 
         "--server.headless", "true", 
         "--server.enableCORS", "false", # 減少一些 cloudflared 連接問題
         "--server.enableXsrfProtection", "false" # 同上，但在公共服務上需謹慎
        ],
        cwd=ACTUAL_PROJECT_DIR,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
        bufsize=1 # Line-buffered
    )
    print(f"Streamlit 程序已啟動 (PID: {streamlit_process.pid})。")

    # 給 Streamlit 一點時間啟動
    time.sleep(5)

    print("正在啟動 cloudflared 隧道...")
    # cloudflared 會輸出幾行，其中一行包含 .trycloudflare.com 的 URL (通常在 stderr)
    tunnel_process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:8501', '--no-autoupdate'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
        bufsize=1 # Line-buffered
    )
    print(f"cloudflared 隧道程序已啟動 (PID: {tunnel_process.pid})。")

    # 創建並啟動輸出監控線程
    st_stdout_thread = threading.Thread(target=stream_output, args=(streamlit_process.stdout, "STREAMLIT_OUT"))
    st_stderr_thread = threading.Thread(target=stream_output, args=(streamlit_process.stderr, "STREAMLIT_ERR"))
    tn_stdout_thread = threading.Thread(target=stream_output, args=(tunnel_process.stdout, "TUNNEL_OUT"))
    # 關鍵：將 is_tunnel_err_pipe 設為 True 以解析 URL
    tn_stderr_thread = threading.Thread(target=stream_output, args=(tunnel_process.stderr, "TUNNEL_ERR", True))

    threads = [st_stdout_thread, st_stderr_thread, tn_stdout_thread, tn_stderr_thread]
    for t in threads: t.start()

    print("Streamlit 服務和 cloudflared 隧道正在背景運行。")
    print("您可以監控上方的日誌輸出。要停止服務，請中斷此儲存格的執行 (Interrupt execution)。")

    # 等待程序結束或中斷
    while True:
        if streamlit_process.poll() is not None:
            print("Streamlit 程序已意外終止。")
            break
        if tunnel_process.poll() is not None:
            print("cloudflared 隧道已意外終止。")
            break
        time.sleep(5) # 每5秒檢查一次

except KeyboardInterrupt:
    print("接收到手動中斷訊號 (KeyboardInterrupt)。正在關閉程序...")
except Exception as e:
    print(f"啟動過程中發生未預期錯誤: {e}", flush=True)
finally:
    print("開始執行清理和終止程序...")
    stop_event.set() # 通知所有線程停止處理新輸出

    if tunnel_process and tunnel_process.poll() is None:
        print("正在嘗試終止 cloudflared 隧道...")
        tunnel_process.terminate() # SIGTERM
        try:
            tunnel_process.wait(timeout=10)
            print("cloudflared 隧道已終止。")
        except subprocess.TimeoutExpired:
            print("cloudflared 隧道終止超時，強制結束 (SIGKILL)...")
            tunnel_process.kill()
            tunnel_process.wait()
            print("cloudflared 隧道已被強制結束。")

    if streamlit_process and streamlit_process.poll() is None:
        print("正在嘗試終止 Streamlit 應用程式...")
        streamlit_process.terminate() # SIGTERM
        try:
            streamlit_process.wait(timeout=10)
            print("Streamlit 應用程式已終止。")
        except subprocess.TimeoutExpired:
            print("Streamlit 應用程式終止超時，強制結束 (SIGKILL)...")
            streamlit_process.kill()
            streamlit_process.wait()
            print("Streamlit 應用程式已被強制結束。")

    print("等待輸出線程結束...")
    for t in threads: # 'threads' 在 try 塊中定義，如果在 try 之前失敗則會出錯
        if t.is_alive():
            t.join(timeout=5)
    print("所有程序和監控線程已結束。如果 Colab 儲存格仍在運行，您可以手動停止它。")